In [1]:
from py5paisa import FivePaisaClient
from py5paisa.order import Order, OrderType, Exchange
import time
import pandas as pd
import warnings
from datetime import datetime

warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
def login():
    cred = {
        "APP_NAME": "5P56409084",
        "APP_SOURCE": "9217",
        "USER_ID": "pybeQPEbjju",
        "PASSWORD": "AESDZQGqyAa",
        "USER_KEY": "542GTEZguBbLERUXRZe26rLvfWU2X2KD",
        "ENCRYPTION_KEY": "l8sRTIsjl1JhUrz8X48XQJzKGI4k0gft",
    }
    client = FivePaisaClient(
        email="vasuappliedai@gmail.com", passwd="Vasudeva@1", dob="19981218", cred=cred
    )
    client.login()

login()

 10:30:59 | Logged in!!


In [3]:
nifty_script_code = 999920000
banknifty_script_code = 999920005
given_date = datetime.today().strftime("%Y-%m-%d")
script_df = pd.read_csv("scripmaster-csv-format.csv",low_memory=False)
bank_nifty_symbol = "BANKNIFTY 12 May 2022"
expiry="20220512"

In [10]:
def rounded_number(number):
    return (round(number / 1000, 1) * 1000).astype(int)

def get_current_hours_mins():
    return datetime.today().hour,datetime.today().minute

In [5]:
def get_current_expiry_strikes(symbol):
    strikePrice = client.historical_data("N", "C", banknifty_script_code, "1m", given_date, given_date)["Open"].iloc[15]
    strikePrice = rounded_number(strikePrice)
    start_strike = strikePrice*0.9
    end_strike = strikePrice*1.1
    temp_df = script_df["FullName"].str.contains(pat=symbol + " CE", regex=True)
    call_df = script_df[temp_df]
    call_df = call_df[call_df["StrikeRate"] > start_strike]
    call_df = call_df[call_df["StrikeRate"] < end_strike]
    call_df.to_csv("bank_nifty_call_expiry.csv")
    temp_df = script_df["FullName"].str.contains(
        pat=symbol + " PE", regex=True)
    put_df = script_df[temp_df]
    put_df = put_df[put_df["StrikeRate"] > start_strike]
    put_df = put_df[put_df["StrikeRate"] < end_strike]
    put_df.to_csv("bank_nifty_put_expiry.csv")
    return call_df, put_df

def map_prices_to_strike_prices(data_frame):
    temp_rows = []
    for _, row in data_frame.iterrows():
        script_code = row["Scripcode"]
        try:
            close_point = client.historical_data("N", "D", script_code, "1m",
                                                        given_date, given_date).tail(1)
            temp_rows.append(close_point["Close"].iloc[0])
        except:
            temp_rows.append(9999)
            continue
    data_frame["LTP"] = temp_rows
    return data_frame.sort_values(by=["LTP"])

def adjust_strangle(option_type,desired_price):
    call_df, put_df = get_current_expiry_strikes(bank_nifty_symbol, strikePrice*0.9, strikePrice*1.1)
    if option_type == "CE":
        call_option_chain = map_prices_to_strike_prices(call_df)
        desired_call_df = call_option_chain[call_option_chain["LTP"]<desired_price].tail(1)
        call_script_code = desired_call_df["Scripcode"].iloc[0]
        call_sell = desired_call_df["LTP"].iloc[0]
        call_symbol = desired_call_df["Name"].iloc[0]
        call_strike = desired_call_df["StrikeRate"].iloc[0]
        print("Sold call call_sell, call_symbol, call_script_code ",call_sell,call_symbol, call_script_code)
        return call_sell,call_symbol,call_script_code,call_strike
    if option_type == "PE":
        put_option_chain = map_prices_to_strike_prices(put_df,entry_point)
        desired_put_df = put_option_chain[put_option_chain["LTP"]<desired_price].tail(1)
        put_script_code = desired_put_df["Scripcode"].iloc[0]
        put_sell = desired_put_df["LTP"].iloc[0]
        put_symbol = desired_put_df["Name"].iloc[0]
        put_strike = desired_put_df["StrikeRate"].iloc[0]
        print("Sold put sell put_sell, put_symbol, put_script_code ",put_sell,put_symbol,put_script_code)
        return put_sell,put_symbol,put_script_code,put_strike

def start_strangle_fn(entry_time,call_script_code,put_script_code):
    call_df = client.historical_data('N','D',call_script_code,'1m',given_date,given_date)
    call_df["Datetime"] = pd.to_datetime(call_df["Datetime"])
    call_df["Close CE"] = call_df["Close"]
    call_df = call_df[call_df["Datetime"]>datetime.fromisoformat(entry_time)]
    call_df = call_df[["Datetime","Close CE"]]
    put_df = client.historical_data('N','D',put_script_code,'1m',given_date,given_date)
    put_df["Datetime"] = pd.to_datetime(put_df["Datetime"])
    put_df["Close PE"] = put_df["Close"]
    put_df = put_df[put_df["Datetime"]>datetime.fromisoformat(entry_time)]
    put_df = put_df[["Datetime","Close PE"]]
    merged_df = pd.merge(call_df,put_df,on="Datetime")
    for index,row in merged_df.iterrows() : 
        call_point = row["Close CE"]
        put_point = row["Close PE"]
        if(call_point*0.5 > put_point):
            print("PE is half of CE at time, call_price and put_price ",row["Datetime"],call_point,put_point)
            adjust_strangle("PE",call_point*0.85,row["Datetime"])
            return "PE",row["Datetime"]
        elif(call_point < put_point*0.5):        
            print("CE is half of PE at index, call_price and put_price ",row["Datetime"],call_point,put_point)
            adjust_strangle("CE",put_point*0.85,row["Datetime"])
            return "CE",row["Datetime"]

def init_strangle():
    call_df, put_df = get_current_expiry_strikes(bank_nifty_symbol)
    call_option_chain = map_prices_to_strike_prices(call_df)
    put_option_chain = map_prices_to_strike_prices(put_df)
    desired_price = 125
    call_point = call_option_chain[call_option_chain["LTP"]<desired_price].tail(1)
    put_point = put_option_chain[put_option_chain["LTP"]<desired_price].tail(1)
    
    call_symbol = call_point["FullName"].iloc[0]
    put_symbol = put_point["FullName"].iloc[0]
    call_script_code = int(call_point["Scripcode"].iloc[0])
    put_script_code = int(put_point["Scripcode"].iloc[0])
    call_strike = call_point["StrikeRate"].iloc[0]
    put_strike = put_point["StrikeRate"].iloc[0]
    call_symbol,put_symbol,call_script_code,put_script_code

    req_list = [
        {
            "Exch": "N",
            "ExchType": "D",
            "Symbol": call_symbol,
            "Expiry": expiry,
            "StrikePrice": call_strike,
            "OptionType": "CE",
        },
        {
            "Exch": "N",
            "ExchType": "D",
            "Symbol": put_symbol,
            "Expiry": expiry,
            "StrikePrice": put_strike,
            "OptionType": "PE",
        },
    ]

    prices = get_price_from_request(req_list)
    call_price = prices[0]
    call_order_id = place_order(order_type="S",script_code=call_script_code,price=call_price)
    put_price = prices[1]
    put_order_id = place_order(order_type="S",script_code=put_script_code,price=call_price)
    
    return call_point,put_point

In [6]:
def get_order_book():
    order_book =client.order_book()
    if order_book is None:
        for i in range(3):
            time.sleep(5)
            order_book =client.order_book()
            if order_book is not None:
                return order_book
    return order_book

def get_order_id(broker_order_id):
    order_book = get_order_book()
    for order in order_book:
        if(order["BrokerOrderId"] == broker_order_id):
            return order["ExchOrderID"]
    return None

def get_price_from_request(req_list):
    data = client.fetch_market_feed(req_list)["Data"]
    if data is None:
        return []
    return list(map( lambda x: x["LastRate"] ,data))
    
def place_order(order_type,script_code,price):
    order = Order(order_type=order_type,exchange='N',exchange_segment='D', scrip_code = script_code, quantity=25, 
                    price=price,is_intraday=False,remote_order_id="tag")
    broker_order_id = client.place_order(order)['BrokerOrderID']
    time.sleep(5)
    return get_order_id(broker_order_id)

In [ ]:
call_point,put_point = init_strangle()

In [11]:
get_current_hours_mins()

(10, 33)

In [39]:
def watch_strangle(call_point,put_point):

    call_symbol = call_point["FullName"].iloc[0]
    put_symbol = put_point["FullName"].iloc[0]
    call_script_code = int(call_point["Scripcode"].iloc[0])
    put_script_code = int(put_point["Scripcode"].iloc[0])
    call_strike = call_point["StrikeRate"].iloc[0]
    put_strike = put_point["StrikeRate"].iloc[0]
    call_symbol,put_symbol,call_script_code,put_script_code

    while hours!=15 and minutes!=20:
        req_list = [
            {
                "Exch": "N",
                "ExchType": "D",
                "Symbol": call_symbol,
                "Expiry": expiry,
                "StrikePrice": call_strike,
                "OptionType": "CE",
            },
            {
                "Exch": "N",
                "ExchType": "D",
                "Symbol": put_symbol,
                "Expiry": expiry,
                "StrikePrice": put_strike,
                "OptionType": "PE",
            },
        ]

        prices = get_price_from_request(req_list)
        call_price = prices[0]
        put_price = prices[1]
        if(call_price <= put_price/2):
            print("Needs Adjustment Here at Call")
            call_sell,call_symbol,call_script_code,call_strike = adjust_strangle("CE",put_price*0.85)
        elif(put_price <= call_price/2):
            print("Needs Adjustment Here at Put")
            put_sell,put_symbol,put_script_code,put_strike = adjust_strangle("PE",call_price*0.85)            
        else:
            time.sleep(60)
        hours, minutes = get_current_hours_mins()

watch_strangle(call_point,put_point)

 23:31:10 | Success
